In [ ]:

def mod_inverse(a, m):
    """Tính nghịch đảo modulo: a^-1 mod m"""
    if a == 0: return 0
    return pow(a, -1, m)

class Curve:
    """Lưu trữ tham số đường cong: y^2 = x^3 + ax + b mod p"""
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p

class Point:
    """Đại diện một điểm (x, y) trên đường cong Elliptic"""
    def __init__(self, curve, x, y):
        self.curve = curve
        self.x = x
        self.y = y

    def is_infinity(self):
        """Kiểm tra xem đây có phải điểm Vô Cực (0) không"""
        return self.x is None and self.y is None

    def __repr__(self):
        if self.is_infinity(): return "Point(Infinity)"
        return f"Point({self.x}, {self.y})"

    def __add__(self, other):
        """Thực hiện phép cộng điểm: P + Q"""
        # 1. Cộng với 0
        if self.is_infinity(): return other
        if other.is_infinity(): return self

        curve = self.curve
        a, p = curve.a, curve.p

        # 2. Cộng đối xứng (x bằng nhau, y khác nhau) -> Ra 0
        if self.x == other.x and self.y != other.y:
            return Point(curve, None, None)

        # 3. Tính Slope (Hệ số góc)
        if self.x == other.x:
            # Trường hợp P == Q (Nhân đôi)
            if self.y == 0: return Point(curve, None, None)
            numerator = (3 * self.x**2 + a) % p
            denominator = (2 * self.y) % p
        else:
            # Trường hợp P != Q (Cộng thường)
            numerator = (other.y - self.y) % p
            denominator = (other.x - self.x) % p

        slope = (numerator * mod_inverse(denominator, p)) % p

        # 4. Tính tọa độ điểm mới
        x3 = (slope**2 - self.x - other.x) % p
        y3 = (slope * (self.x - x3) - self.y) % p

        return Point(curve, x3, y3)

    def __mul__(self, scalar):
        """Thực hiện phép nhân điểm: k * P (Double-and-Add)"""
        result = Point(self.curve, None, None) # Khởi tạo là 0
        addend = self

        while scalar > 0:
            if scalar % 2 == 1:
                result = result + addend
            
            addend = addend + addend
            scalar = scalar // 2
            
        return result

# --- TEST DRIVE (KIỂM CHỨNG TÍNH TOÁN) ---
if __name__ == "__main__":
    print("--- KIỂM TRA PHÉP TÍNH ECC ---")
    
    # 1. Thiết lập đường cong (Ví dụ trong bài giảng Giai đoạn 1)
    # y^2 = x^3 + 2x + 2 mod 17
    curve_17 = Curve(a=2, b=2, p=17)
    
    # 2. Điểm P(5, 1)
    P = Point(curve_17, 5, 1)
    print(f"Điểm P: {P}")
    
    # 3. Tính 2P (Nhân đôi)
    # Theo tính toán tay ở Giai đoạn 1: 2P phải ra (6, 3)
    P2 = P + P
    print(f"2 * P : {P2}")
    
    # 4. Tính bằng phép nhân (Scalar Multiplication)
    P2_mul = P * 2
    print(f"P * 2 : {P2_mul}")
    
    if P2.x == 6 and P2.y == 3 and P2_mul.x == 6:
        print("=> CHÍNH XÁC! Phép tính trùng khớp với lý thuyết. ✅")
    else:
        print("=> SAI! Kiểm tra lại code. ❌")

    # 5. Thử với số lớn hơn (Ví dụ 93 * P xem nó ra điểm nào)
    # Đây là cách tạo Public Key từ Private Key 93
    public_key = P * 93
    print(f"Public Key (93 * P) = {public_key}")

In [8]:
def mod_inverse(a, m):
    #tinh nghich dao cua a^-1 mod m
    #dkien a != 0
    if a == 0: return 0
    return pow(a, -1, m)

class Curve:
    
    #ưu trữ  đường cong: y^2 = x^3 + ax + b mod p
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p

class Point:
    
    # luu tru toa di cac diem
    def __init__(self, curve, x, y):
        self.curve = curve
        self.x = x
        self.y = y

    def is_infinity(self):
        return self.x is None and self.y is None
    
    def __repr__(self):
        # Hàm này để in ra màn hình cho đẹp: (x, y)
        if self.is_infinity(): return "Point(Infinity)"
        return f"Point({self.x}, {self.y})"
    
    # xu li diem vo cuc ( neu a = 4, b =0 thi a +b =a va nguoc lai)
    def __add__(self, other):
        if self.is_infinity(): return other
        if other.is_infinity(): return self
        
        curve = self.curve
        a, p =self.curve.a, self.curve.p
        
        # neu a =4, b= -4 => a +b =0
        if self.x == other.x and self.y != other.y:
            return Point(self.curve, None, None)
        
        # tinh he so goc
        if self.x == other.x:
            # Trường hợp P == Q (Nhân đôi - Tiếp tuyến)
            # Công thức: (3x^2 + a) / 2y
            if self.y == 0: return Point(self.curve, None, None)
            
            #truong hop P == Q
            numerator = (3 * self.x**2 + a) % p
            denominator = (2 * self.y) % p
            
        else:
            # P != Q
            numerator = (other.y -self.y) % p
            denominator = (other.x -self.x) % p
        # he so goc  
        slope = (numerator * mod_inverse(denominator, p)) % p
        
        #toa do diem moi
        x3 = (slope**2 -self.x - other.x) % p 
        y3 = (slope *(self.x- x3)- self.y)% p

        return Point(curve, x3, y3)
    
    def __mul__(self, scalar):
        
        #thuc hien phep nhan diem: k* p
        result = Point(self.curve, None, None) # khoi tao = 0
        addend = self
        
        while scalar >0: 
            if scalar % 2 ==1:
                result = result + addend
            
            addend = addend + addend
            scalar = scalar // 2
        return result

if __name__ == "__main__":
    print("--- KIỂM TRA PHÉP TÍNH ECC ---")
    
    # 1. Thiết lập đường cong (Ví dụ trong bài giảng Giai đoạn 1)
    # y^2 = x^3 + 2x + 2 mod 17
    curve_17 = Curve(a=2, b=2, p=17)
    
    # 2. Điểm P(5, 1)
    P = Point(curve_17, 5, 1)
    print(f"Điểm P: {P}")
    
    # 3. Tính 2P (Nhân đôi)
    # Theo tính toán tay ở Giai đoạn 1: 2P phải ra (6, 3)
    P2 = P + P
    print(f"2 * P : {P2}")
    
    # 4. Tính bằng phép nhân (Scalar Multiplication)
    P2_mul = P * 2
    print(f"P * 2 : {P2_mul}")
    
    if P2.x == 6 and P2.y == 3 and P2_mul.x == 6:
        print("=> CHÍNH XÁC! Phép tính trùng khớp với lý thuyết. ")
    else:
        print("=> SAI! Kiểm tra lại code. ")

    # 5. Thử với số lớn hơn (Ví dụ 93 * P xem nó ra điểm nào)
    # Đây là cách tạo Public Key từ Private Key 93
    public_key = P * 93
    print(f"Public Key (93 * P) = {public_key}")    

--- KIỂM TRA PHÉP TÍNH ECC ---
Điểm P: Point(5, 1)
2 * P : Point(6, 3)
P * 2 : Point(6, 3)
=> CHÍNH XÁC! Phép tính trùng khớp với lý thuyết. 
Public Key (93 * P) = Point(6, 14)
